In [1]:
from model.rvae import RVAE
from util.batch_loader import Batch
from util.preprocess import Preprocess
from util.parameter import Parameter
from gensim.models import KeyedVectors
from torch.optim import Adam
import numpy as np
import torch

### load embedding model

In [2]:
embedding_model=KeyedVectors.load_word2vec_format('embedding.bin')

### load raw data

In [3]:
with open('train.txt','r') as f:
    data=f.readlines()

### preprocess

In [4]:
import os
preprocess=Preprocess(embedding_model)
input=preprocess.to_sequence(data)
if not os.path.exists('embedding.npy'):
    embedding=preprocess.embedding()
    np.save('embedding',embedding)

### batch generator and parameter

In [5]:
batch_loader=Batch(input,0.7)
params=Parameter(word_embed_size=300,encode_rnn_size=600,latent_variable_size=1400,\
            decode_rnn_size=600,vocab_size=preprocess.vocab_size,embedding_path='embedding.npy')

### model , optimizer and train

In [10]:
model=RVAE(params)
optimizer=Adam(model.learnable_parameters(), 5e-5)
train_step=model.trainer(optimizer)
if os.path.exists('trained_rvae'):
    model.load_state_dict(torch.load('trained_rvae'))
use_cuda=torch.cuda.is_available()
if use_cuda:
    model=model.cuda()
ce_list=[]
kld_list=[]
coef_list=[]

In [11]:
print(model)

RVAE (
  (encoder): Encoder (
    (highway): Highway (
    )
    (lstm): LSTM(300, 600, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder (
    (lstm): LSTM(1700, 600, batch_first=True)
    (fc): Linear (600 -> 18803)
  )
  (logvar): Linear (1200 -> 1400)
  (mu): Linear (1200 -> 1400)
  (embedding): Embedding (
    (word_embed): Embedding(18803, 300)
  )
)


In [12]:
use_cuda

True

### train

In [ ]:
for i,batch in enumerate(batch_loader.train_next_batch(64)):

#     if i%21==0:
#         seed = np.random.normal(size=[1, params.latent_variable_size])
#         sentence=model.sample(50,seed,use_cuda)
#         sentence=[preprocess.index_to_word[i] for i in sentence]
#         print(' '.join(sentence))
#         continue
    ce,kld,coef=train_step(i,batch,300,0.2,use_cuda)
    if i%20==0:
        print('ten step: ce:{}, kld:{} ,coef:{}'.format(ce,kld,coef))
        torch.save(model.state_dict(), 'trained_rvae')
    ce_list+=[ce]
    kld_list+=[kld]
    coef_list+=[coef]
print("epoch finish")

ten step: ce:
 9.9050
[torch.cuda.FloatTensor of size 1 (GPU 0)]
, kld:
 7.9184e+29
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 ,coef:0.0009110511944006583
ten step: ce:
 9.7431
[torch.cuda.FloatTensor of size 1 (GPU 0)]
, kld:
 3.8488e+35
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 ,coef:0.0009481966443537493
ten step: ce:
nan
[torch.cuda.FloatTensor of size 1 (GPU 0)]
, kld:
nan
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 ,coef:0.0009868550955209332


KeyboardInterrupt: 